#IR spectral analysis of organic and coordination compounds via machine learning and neural networks
By: Koshelev Daniil 

-Work in progress (last updated:10/06/2023)-

B. Цель данного блокнота

Целью этого блокнота является создание датасета на 92 класса и обучение модели на классификацию 92 классов.

C. План реализации

1. Импортировать данные
2. Реализовать приписывание 92 классов через правильные SMARTs ключи
3. Создать класс датасета
4. Создать и обучить модель
5. Собрать статистику

NOTES
1. Импортированы данные
2. Создан словарь SMARTs ключей для разметки на 92 и 72 класса
3. Реализован эмпирический алгоритм для 72 класса
4. Оценена точность подходов классического машинного обучения 


Клонирование данных с GitHub. На сайте выложены как исходные списки веществ, так и скачанные спектры соединений, которые были запарсены кодом ниже.

In [1]:
!git clone https://github.com/Lamblador/IR_expert_system.git

Cloning into 'IR_expert_system'...
remote: Enumerating objects: 17276, done.
remote: Counting objects: 100% (2433/2433), done.
remote: Compressing objects: 100% (2272/2272), done.
remote: Total 17276 (delta 179), reused 2397 (delta 157), pack-reused 14843
Receiving objects: 100% (17276/17276), 1.47 GiB | 27.16 MiB/s, done.
Resolving deltas: 100% (539/539), done.
Updating files: 100% (16977/16977), done.


In [2]:
from IPython.core.display import clear_output
!pip install --upgrade jcamp
!pip install cirpy
!pip install peakutils
!pip install datamol
!pip install rdkit
clear_output()

In [3]:
# Вводятся нужные импорты
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import os
import numpy as np
import torch
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets
import torch.nn as nn
from peakutils.peak import indexes as index_utils


In [4]:
import random

import torch
import torch.nn as nn
from torchvision import datasets, transforms


def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

set_random_seed(42)

# Импорт табличных данных

In [14]:
labels72_df = pd.read_csv('/content/IR_expert_system/data/nist_ir_spectra_list_with_72_bonds.csv')
labels72_df = labels72_df.set_index("CAS")

In [15]:
#Считывание датасета
def read_dataset(dataset_name):
  '''
  Function for reading dataset into pandas Dataframe from /IR_expert_system/data/datasets/

  dataset_name - name of dataset's folder to read

  return pd.Dataframe with whole dataset

  parameters:
      name = ['dataset3']
          dataset3 - dataset made from reading most of jcam files, with padded y, interpolated x and peak layer. Saved in 10 pieces. 
  '''
  df_list = []
  # choose current directory
  curr_directory = os.getcwd()
  os.chdir(curr_directory)  # set to current directory
  files = glob.glob('/content/IR_expert_system/data/datasets/'+dataset_name+'/*.pkl')  # look for files without extension. Returns a list.
  # Try to read the file. If can't be read, skip it.
  #print(files)
  for i, file in tqdm(enumerate(files)):
      df = pd.read_pickle(file)
      df_list.append(df)
  #print(df_list)
  result = pd.concat(df_list)
  return result

dataset_df1 = read_dataset('dataset3')

10it [00:03,  2.57it/s]


In [16]:
dataset72_df_labeled = labels72_df.join(dataset_df1)
dataset72_df_labeled

,Unnamed: 0,Name,Formula,Mw,Elements,InChi,SMILES,C─H alkyl methyl 1260 strong/1380 weak/2870 medium to strong/2960 medium to strong,C─H alkyl methylene 1470 strong/2850 medium to strong/2925 medium to strong,C─H alkyl methine 2890 weak,...,number of interferograms averaged per single channel spectrum,sample description,temperature,$spectra version,$uncertainty in y,pressure,x1,y1,y_peak,x_peak
CAS,,,,,,,,,,,,,,,,,,,,,
100-00-5,0,"Benzene, 1-chloro-4-nitro-",C6H4ClNO2,156.993056,"['C', 'H', 'Cl', 'N', 'O']",InChI=1/C6H4ClNO2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H,[O-][N+](=O)c1ccc(Cl)cc1,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.026076922193169594, 0.02797572687268257, 0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0999999999999...","[530, 742, 850, 1014, 1102, 1174, 1274, 1350, ..."
100-01-6,1,p-Nitroaniline,C6H6N2O2,138.042927,"['C', 'H', 'N', 'O']",InChI=1/C6H6N2O2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H...,Nc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.005049999803304672, 0.005049999803304672, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[624, 684, 744, 838, 1114, 1180, 1290, 1346, 1..."
100-02-7,2,"Phenol, 4-nitro-",C6H5NO3,139.026943,"['C', 'H', 'N', 'O']","InChI=1/C6H5NO3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H",Oc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0002783999952953309, 0.0002783999952953309,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[626, 746, 858, 1110, 1186, 1270, 1350, 1438, ..."
1000-49-3,5,N-butyltrimethylsilane,C7H18Si,130.117777,"['C', 'H', 'Si']","InChI=1/C7H18Si/c1-5-6-7-8(2,3)4/h5-7H2,1-4H3",CCCC[Si](C)(C)C,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.08899998664855957, 0.08958601189397797, 0.0...","[0.7, 0.8, 0.9, 1.0, 0.9, 0.8, 0.7, 0.6, 0.5, ...","[506, 574, 606, 644, 686, 724, 750, 828, 856, ..."
1000-50-6,6,N-Butyldimethylchlorosilane,C6H15ClSi,150.063155,"['C', 'H', 'Cl', 'Si']","InChI=1/C6H15ClSi/c1-4-5-6-8(2,3)7/h4-6H2,1-3H3",CCCC[Si](C)(C)Cl,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.20800000429153442, 0.20800000429153442, 0.2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[662, 712, 756, 792, 814, 850, 880, 966, 1006,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99-97-8,44445,"Benzenamine, N,N,4-trimethyl-",C9H13N,135.104799,"['C', 'H', 'N']","InChI=1/C9H13N/c1-8-4-6-9(7-5-8)10(2)3/h4-7H,1...",CN(C)c1ccc(C)cc1,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.02355931419879198, 0.02319779060781002, 0.0...","[0.0, 0.0, 0.09999999999999998, 0.199999999999...","[522, 806, 950, 1062, 1158, 1218, 1262, 1338, ..."
999-78-0,44446,"2-Pentyne, 4,4-dimethyl-",C7H12,96.093900,"['C', 'H']","InChI=1/C7H12/c1-5-6-7(2,3)4/h1-4H3",CC#CC(C)(C)C,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.0013761000009253621, 0.0013761000009253621,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1202, 1270, 1370, 1462, 2878, 2934, 2974]"
999-81-5,44448,"Ammonium chloride, 2-chloroethyltrimethyl-",C5H13ClN.Cl,157.042505,"['C', 'H', 'Cl', 'N', 'Cl']","InChI=1/C5H13ClN.ClH/c1-7(2,3)5-4-6;/h4-5H2,1-...",[Cl-].C[N+](C)(C)CCCl,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"[500, 502, 504, 506, 508, 510, 512, 514, 516, ...","[0.13220000267028809, 0.1312999725341797, 0.12...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[666, 730, 768, 804, 920, 950, 978, 1062, 1140..."


In [17]:
df_colums_name_list1 = ['Name', 'Formula', 'Mw', 'Elements', 'InChi', 'SMILES', 'title', 'molform']
df_colums_name_list2= ['x', 'y', 'x1', 'y1', 'y_peak']

In [18]:
dataset72_df_labeled1 = dataset72_df_labeled[[
 'Name',
 #'Formula',
 #'Mw',
 #'Elements',
 'InChi',
 #'SMILES',
 'C─H alkyl methyl 1260 strong/1380 weak/2870 medium to strong/2960 medium to strong',
 'C─H alkyl methylene 1470 strong/2850 medium to strong/2925 medium to strong',
 'C─H alkyl methine 2890 weak',
 'C─H vinyl C═CH2 900 strong/2975 medium/3080 medium',
 'C─H vinyl C═CH 3020 medium',
 'C─H vinyl monosubstituted alkenes 900 strong/990 strong',
 'C─H vinyl cis-disubstituted alkenes 670–700 strong',
 'C─H vinyl trans-disubstituted alkenes 965 strong',
 'C─H vinyl trisubstituted alkenes 800–840 strong to medium',
 'C─H aromatic benzene/sub. benzene 3070 weak',
 'C─H aromatic monosubstituted benzene 700–750 strong',
 'C─H aromatic monosubstituted benzene 690–710 strong',
 'C─H aromatic ortho-disub. benzene 750 strong',
 'C─H aromatic meta-disub. benzene 750–800 strong',
 'C─H aromatic meta-disub. benzene 860–900 strong',
 'C─H aromatic para-disub. benzene 800–860 strong',
 'C─H alkynes any 3300 medium',
 'C─H aldehydes any 2720 medium',
 'C─H aldehydes any 2820 medium',
 'C═C acyclic C═C monosub. alkenes 1645 medium',
 'C═C acyclic C═C 1,1-disub. alkenes 1655 medium',
 'C═C acyclic C═C cis-1,2-disub. alkenes 1660 medium',
 'C═C acyclic C═C trans-1,2-disub. alkenes 1675 medium',
 'C═C acyclic C═C trisub., tetrasub. alkenes 1670 weak',
 'C═C conjugated C═C dienes 1600 strong/1650 strong',
 'C═C with benzene ring dienes 1625 strong',
 'C═C with C═O dienes 1600 strong',
 'C═C C═C (both sp2) any 1640–1680 medium',
 'C═C aromatic C═C any 1450/1500/1580/1600 weak to strong (usually 3 or 4)',
 'C═C C≡C terminal alkynes 2100–2140 weak',
 'C═C C≡C disubst. alkynes 2190–2260 very weak (often indistinguishable)',
 'C=O aldehyde/ketone saturated aliph./cyclic 6-membered 1720 ',
 'C=O aldehyde/ketone α,β-unsaturated 1685 ',
 'C=O aldehyde/ketone aromatic ketones 1685 ',
 'C=O aldehyde/ketone aldehydes 1725 influenced by conjugation (as with ketones)',
 'C=O carboxylic acids/derivates saturated carboxylic acids 1710 ',
 'C=O carboxylic acids/derivates unsat./aromatic carb. acids 1680–1690 ',
 'C=O carboxylic acids/derivates esters and lactones 1735 influenced by conjugation and ring size (as with ketones)',
 'C=O carboxylic acids/derivates anhydrides 1760 ',
 'C=O carboxylic acids/derivates anhydrides 1820 ',
 'C=O carboxylic acids/derivates acyl halides 1800 ',
 'C=O carboxylic acids/derivates amides 1650 associated amides',
 'C=O carboxylic acids/derivates amino acid zwitterions 1550–1610 ',
 'O─H alcohols, phenols low concentration 3610–3670/high concentration 3200–3400 broad',
 'O─H carboxylic acids low concentration 3500–3560/high concentration 3000 broad',
 'N─H primary amines any 3400–3500 strong/1560–1640 strong',
 'N─H secondary amines any > 3000 weak to medium',
 'N─H ammonium ions any 2400–3200 multiple broad peaks',
 'C─O alcohols primary 1040–1060 strong, broad',
 'C─O alcohols secondary ~ 1100 strong',
 'C─O alcohols tertiary 1150–1200 medium',
 'C─O phenols any 1200',
 'C─O ethers aliphatic 1120',
 'C─O ethers aromatic 1220–1260',
 'C─O carboxylic acids any 1250–1300',
 'C─O esters any 1100–1300 two bands (distinct from ketones, which do not possess a C─O bond)',
 'C─N aliphatic amines any 1020–1220 often overlapped',
 'C─N C═N any 1615–1700 similar conjugation effects to C═O',
 'C─N C≡N (nitriles) unconjugated 2250 medium',
 'C─N C≡N (nitriles) conjugated 2230 medium',
 'C─N R─N─C (isocyanides) any 2165–2110',
 'C─N R─N═C═S (isothiocyanates) any 2140–1990',
 'C─X fluoroalkanes ordinary 1000–1100',
 'C─X fluoroalkanes trifluoromethyl 1100–1200 two strong, broad bands',
 'C─X chloroalkanes any 540–760 weak to medium',
 'C─X bromoalkanes any 500–600 medium to strong',
 'C─X iodoalkanes any 500 medium to strong',
 'N─O nitro compounds aliphatic 1540 stronger/aliphatic 1380 weaker',
 'N─O nitro compounds aromatic 1520 lower/1350 lower if conjugated',
 'P─C Organophosphorus compound aromatic 1440-1460 medium',
 'P─O phosphorus oxide bonded 1195-1250 strong',
 'P─O phosphorus oxide free 1250-1300 strong',
#  'index',
#  'title',
#  'jcamp-dx',
#  'data type',
#  'class',
#  'origin',
#  'owner',
#  'date',
#  'names',
#  'molform',
#  'source reference',
#  '$nist source',
#  '$nist image',
#  'spectrometer/data system',
#  'path length',
#  'sampling procedure',
#  'resolution',
#  'data processing',
#  'xunits',
#  'yunits',
#  'xfactor',
#  'yfactor',
#  'deltax',
#  'firstx',
#  'lastx',
#  'firsty',
#  'maxx',
#  'minx',
#  'maxy',
#  'miny',
#  'npoints',
#  'xydata',
#  'end',
 'x',
 'y',
#  'filename',
#  'state',
#  'instrument parameters',
#  'xlabel',
#  'ylabel',
#  'mp',
#  'bp',
#  '$nist doc file',
#  '$nist psd file',
#  'aperture',
#  'external diffuse reflectance accessory',
#  'beamsplitter',
#  'detector (dia. det. port in sphere)',
#  'sphere diameter',
#  'acquisition mode',
#  'scanner speed',
#  'coadded scans',
#  'phase resolution',
#  'phase correction',
#  'zerofilling',
#  'spectral range',
#  'spectral resolution',
#  'wavenumber accuracy',
#  'apodization function',
#  'low pass filter',
#  'switch gain on',
#'cas name',
#  'instrument resolution',
#  'ir source',
#  'detector',
#  'interferogram zerofill',
#  'spectral interval after zerofilling',
#  'apodization',
#  'folding limits',
#  'number of interferograms averaged per single channel spectrum',
#  '$spectra version',
#  '$uncertainty in y',
#  'sample description',
#  'pressure',
#  'temperature',
 'x1',
 'y1',
 'y_peak',
 'x_peak']]

#Эмпирический алгоритм соответствия

Создание списка диапазонов проявления классов

In [23]:
#Список полос колебаний
all_num_list = [
[1260,1380,2870,2960],
[1470,2850,2925],
[2890],
[2900,2975,3080],
[3020],
[900,990],
[670,700],
[965],
[800,840],
[3070],
[700,750],
[690,710],
[750],
[750,800],
[860,900],
[800,860],
[3300],
[2720],
[2820],
[1645],
[1655],
[1660],
[1675],
[1670],
[1600, 1650],
[1625],
[1600],
[1640,1680],
[1450,1500,1580,1600],
[2100,2140],
[2190,2260],
[1720],
[1685],
[1685],
[1725],
[1710],
[1680,1690],
[1735],
[1760],
[1820],
[1800],
[1650],
[1550,1610],
[3610,3670, 3200,3400],
[3500,3560, 3000],
[3400,3500, 1560,1640],
[3000],
[2400,3200],
[1040,1060],
[1100],
[1150,1200],
[1200],
[1120],
[1220,1260],
[1250,1300],
[1100,1300],
[1020,1220],
[1615,1700],
[2250],
[2230],
[2165,2110],
[2140,1990],
[1000,1100],
[1100,1200],
[540,760],
[500,600],
[500],
[1540,1380],
[1520,1350],
[1440,1460],
[1195,1250],
[1250,1300],
]

In [24]:
def det_nearest_peak_ind(value, peak_list):
  min = 10000
  min_ind = 73
  for i, peak in enumerate(peak_list):
    for band in peak:
      temp = abs(band-value)
      if temp<min: 
        min = temp
        min_ind = i
  return min_ind
det_nearest_peak_ind(1195, all_num_list)

70

In [25]:
cls_serie = []
for peak_list in tqdm(dataset72_df_labeled1['x_peak']):
  cls_list = []
  try:
    for value in peak_list:
      cls_list.append(det_nearest_peak_ind(value, all_num_list))
    cls_serie.append(cls_list)
  except:
    cls_serie.append(np.nan)
print(cls_serie)

100%|██████████| 12233/12233 [00:14<00:00, 816.80it/s] 

[[64, 10, 8, 56, 49, 70, 0, 68, 69, 1, 67, 24], [65, 11, 10, 8, 52, 70, 54, 68, 1, 67, 25, 43], [65, 10, 14, 49, 70, 0, 68, 69, 1, 28, 67, 24, 43], [65, 65, 65, 6, 11, 11, 10, 8, 14, 7, 62, 48, 49, 50, 70, 54, 54, 68, 0, 0, 69, 68, 28, 27, 32, 35, 38, 39, 61, 61, 29, 47, 47, 47, 47, 47, 47, 17, 17, 0, 1, 0, 43, 43, 43, 43], [6, 11, 64, 8, 8, 8, 5, 7, 62, 56, 49, 50, 0, 54, 68, 0, 69, 1, 28, 42, 21, 3, 44], [65, 8, 7, 48, 70, 54, 54, 68, 69, 69, 1, 28, 24, 57, 1, 0, 4], [5, 48, 70, 0, 0, 28, 27, 17, 1, 3, 3], [65, 49, 70, 0, 28, 0, 1, 3, 9], [65, 10, 8, 7, 48, 49, 50, 53, 54, 68, 69, 1, 68, 42, 35, 17, 18, 0, 3, 1, 44, 4, 3], [65, 11, 8, 14, 49, 70, 53, 68, 67, 24], [11, 10, 64, 8, 8, 5, 7, 56, 48, 49, 70, 54, 68, 1, 68, 42, 32, 31, 40, 39, 61, 59, 30, 47, 47, 17, 17, 18, 2, 3, 9, 43, 16, 43, 44, 44, 43, 43], [11, 8, 14, 56, 49, 70, 0, 68, 28, 67, 42], [50, 68, 69, 34, 0, 1], [65, 6, 11, 11, 64, 8, 8, 14, 5, 5, 7, 48, 49, 54, 0, 28, 28, 25, 20, 31, 17, 18, 18, 0, 1, 44, 4, 43, 44], [6, 

In [27]:
cls_serie1 = pd.Series(cls_serie)
df = pd.DataFrame(cls_serie1, columns = ['peak_cls'])
dataset72_df_labeled1.reset_index()
dataset72_df = dataset_df1.reset_index()
dataset72_df1 = dataset72_df.join(df, lsuffix='x')
dataset72_df1 = dataset72_df1.set_index('cas registry no')


In [ ]:
dataset72_df1['peak_cls'].dropna()

#Приписание метки ближайшего соседа

In [28]:
#Сокращённый словарь SMARTS для 72 классов
all_bonds_types_strs_short = {
'C─H alkyl methyl 1260 strong/1380 weak/2870 medium to strong/2960 medium to strong': '[CH3]',
'C─H alkyl methylene 1470 strong/2850 medium to strong/2925 medium to strong': '[CX4;H2]',
'C─H alkyl methine 2890 weak': '[CX4;H1]',
'C─H vinyl C═CH2 900 strong/2975 medium/3080 medium': '[CX3]=[CH2]',
'C─H vinyl C═CH 3020 medium': '[CX3]=[CX3;H1]',
'C─H vinyl monosubstituted alkenes 900 strong/990 strong': '[CX3;H2]=[CX3;H1]',
'C─H vinyl cis-disubstituted alkenes 670–700 strong': '[!H]\[CX3;H1]=[CX3;H1]/[!H]', #Проверить работу цис-транс-изомерии
'C─H vinyl trans-disubstituted alkenes 965 strong': '[!H]/[CX3;H1]=[CX3;H1]/[!H]',
'C─H vinyl trisubstituted alkenes 800–840 strong to medium': '[CX3;H1]=[CX3;H0]',
'C─H aromatic benzene/sub. benzene 3070 weak': '[c;H1]',
'C─H aromatic monosubstituted benzene 700–750 strong': '[c;!H1]1[c;H1][c;H1][c;H1][c;H1][c;H1]1',
'C─H aromatic monosubstituted benzene 690–710 strong': '[c;!H1]1[c;H1][c;H1][c;H1][c;H1][c;H1]1',
'C─H aromatic ortho-disub. benzene 750 strong': '[c;!H1]1[c;H1][c;H1][c;H1][c;H1][c;!H1]1',
'C─H aromatic meta-disub. benzene 750–800 strong': '[c;!H1]1[c;H1][c;H1][c;H1][c;!H1][c;H1]1',
'C─H aromatic meta-disub. benzene 860–900 strong': '[c;!H1]1[c;H1][c;H1][c;H1][c;!H1][c;H1]1',
'C─H aromatic para-disub. benzene 800–860 strong': '[c;!H1]1[c;H1][c;H1][c;!H1][c;H1][c;H1]1',
'C─H alkynes any 3300 medium': '[CH1]#C',
'C─H aldehydes any 2720 medium': '[CX3H1](=O)',
'C─H aldehydes any 2820 medium': '[CX3H1](=O)',
'C═C acyclic C═C monosub. alkenes 1645 medium': '[CX3;H1]=[CH2]',
'C═C acyclic C═C 1,1-disub. alkenes 1655 medium': '[CX3;!H1,!H2]=[CH2]',
'C═C acyclic C═C cis-1,2-disub. alkenes 1660 medium': '*\[CX3;!H2,H1]=[CX3;!H2,H1]/*',#Проверить работу цис-транс-изомерии
'C═C acyclic C═C trans-1,2-disub. alkenes 1675 medium': '*\[CX3;!H2,H1]=[CX3;!H2,H1]/*',#Проверить работу цис-транс-изомерии
'C═C acyclic C═C trisub., tetrasub. alkenes 1670 weak': ['[CX3]=[CX3;H0]','[CX3;H0]=[CX3;H0]'],
'C═C conjugated C═C dienes 1600 strong/1650 strong': '[CX3]=[CX3][CX3]=[CX3]', 
'C═C with benzene ring dienes 1625 strong': '[CX3]=[CX3][r6]',
'C═C with C═O dienes 1600 strong': '[CX3]=[CX3][CX3]=[O]',
'C═C C═C (both sp2) any 1640–1680 medium': '[CX3]=[CX3]',
'C═C aromatic C═C any 1450/1500/1580/1600 weak to strong (usually 3 or 4)': 'cc',
'C═C C≡C terminal alkynes 2100–2140 weak': '[CH1]#C',
'C═C C≡C disubst. alkynes 2190–2260 very weak (often indistinguishable)': '[CD2]#[CD2]', #Проверить работоспособность
'C=O aldehyde/ketone saturated aliph./cyclic 6-membered 1720 ': ['[#6][CX3](=O)[#6]', '[CX3H1](=O)[#6]', '[r6][CX3](=O)', '[r6][CX3H1](=O)'],
'C=O aldehyde/ketone α,β-unsaturated 1685 ': ['[CX3]=[CX3][CX3](=O)[#6]','[CX3]=[CX3][#6][CX3](=O)[#6]', '[CX3]=[CX3][CX3H1](=O)','[CX3]=[CX3][#6][CX3H1](=O)'],
'C=O aldehyde/ketone aromatic ketones 1685 ': 'c[CX3](=O)',
'C=O aldehyde/ketone aldehydes 1725 influenced by conjugation (as with ketones)': '[#6][CX3H1](=O)',
'C=O carboxylic acids/derivates saturated carboxylic acids 1710 ': '[!$(*#C),!$(*=C)][CX3](=O)[OX2H1]', #Проверить работоспособность
'C=O carboxylic acids/derivates unsat./aromatic carb. acids 1680–1690 ': '[$(*#C),$(*=C),c][CX3](=O)[OX2H1]',
'C=O carboxylic acids/derivates esters and lactones 1735 influenced by conjugation and ring size (as with ketones)': ['[#6][CX3](=O)[OX2H0][#6]', '[r][CX3](=O)[OX2H0]'],
'C=O carboxylic acids/derivates anhydrides 1760 ': '[CX3](=O)[OX2H0][CX3](=O)', 
'C=O carboxylic acids/derivates anhydrides 1820 ': '[CX3](=O)[OX2H0][CX3](=O)',
'C=O carboxylic acids/derivates acyl halides 1800 ': '[CX3](=[OX1])[F,Cl,Br,I]',
'C=O carboxylic acids/derivates amides 1650 associated amides': '[NX3][CX3](=[OX1])[#6]',
'C=O carboxylic acids/derivates amino acid zwitterions 1550–1610 ': ['[NH2][CX4][CX3](=O)[OX2H1]', '[NH3+][CX4][CX3](=O)[OX1-]'],
'O─H alcohols, phenols low concentration 3610–3670/high concentration 3200–3400 broad': '[r6][OX2H1]',
'O─H carboxylic acids low concentration 3500–3560/high concentration 3000 broad': '[CX3](=O)[OX2H1]',
'N─H primary amines any 3400–3500 strong/1560–1640 strong': '[NX3H2]',
'N─H secondary amines any > 3000 weak to medium': '[NX3H1]',
'N─H ammonium ions any 2400–3200 multiple broad peaks': '[NH3+]',
'C─O alcohols primary 1040–1060 strong, broad': '[CX4H2][OX2H1]',
'C─O alcohols secondary ~ 1100 strong': '[CX4H1][OX2H1]',
'C─O alcohols tertiary 1150–1200 medium': '[CX4H0][OX2H1]',
'C─O phenols any 1200': '[r6][OX2H1]',
'C─O ethers aliphatic 1120': '[CX4][OX2H0][CX4]',
'C─O ethers aromatic 1220–1260': '[c][OX2H0][c]',
'C─O carboxylic acids any 1250–1300': '[CX3](=O)[OX2H1]',
'C─O esters any 1100–1300 two bands (distinct from ketones, which do not possess a C─O bond)': '[#6][CX3](=O)[OX2H0][#6]',
'C─N aliphatic amines any 1020–1220 often overlapped': '[C;!c][NX3]',
'C─N C═N any 1615–1700 similar conjugation effects to C═O': '[C](=N)',
'C─N C≡N (nitriles) unconjugated 2250 medium': '[C](#N)',
'C─N C≡N (nitriles) conjugated 2230 medium': '[CX3]=[CX3][C](#N)',
'C─N R─N─C (isocyanides) any 2165–2110': ['[C-]#[N+][*]', '[C]=[N][*]'],
'C─N R─N═C═S (isothiocyanates) any 2140–1990': '[S]=[C]=[N][*]',
'C─X fluoroalkanes ordinary 1000–1100': '[CX4](F)',
'C─X fluoroalkanes trifluoromethyl 1100–1200 two strong, broad bands': '[CX4](F)(F)(F)',
'C─X chloroalkanes any 540–760 weak to medium': '[CX4](Cl)',
'C─X bromoalkanes any 500–600 medium to strong': '[CX4](Br)',
'C─X iodoalkanes any 500 medium to strong': '[CX4](I)',
'N─O nitro compounds aliphatic 1540 stronger/aliphatic 1380 weaker': ['[C][NX3+](=O)[OX1-]','[C][NX3](=O)[OX1]'],
'N─O nitro compounds aromatic 1520 lower/1350 lower if conjugated': ['[c][NX3+](=O)[OX1-]','[c][NX3](=O)[OX1]'],
'P─C Organophosphorus compound aromatic 1440-1460 medium': '[P][C]',
'P─O phosphorus oxide bonded 1195-1250 strong': '[P][O][*]',
'P─O phosphorus oxide free 1250-1300 strong': '[P][O]',
}

In [29]:
classes = []
for key, value in all_bonds_types_strs_short.items():
  classes.append(key)
len(classes)

72

In [32]:
# Number of classes
num_classes = len(classes)
num_classes
cls2inx = {}
inx2cls= {}
for i, classs in enumerate(classes):
  cls2inx[f'{classs}']= i
  inx2cls[i] = classs
#print(cls2inx)
#print(inx2cls)

In [33]:
#Функция выдаёт список меток классов по набору индексов классов из 72
#Нужно перевести индексы классов в набор 1 и 0
def find_cls_labels(peak_list, cls2indx_bonds=cls2inx, indx2cls_bonds=inx2cls, fuc_grp_bonds_types=all_bonds_types_strs_short):
    mol_func_grps = []
    for cls, ind in cls2indx_bonds.items():
      prescence = 0
      if ind in peak_list: prescence=1
      mol_func_grps.append(prescence)  
    return mol_func_grps

In [34]:
#Проверка функции
len(find_cls_labels(dataset72_df1['peak_cls'].dropna().iloc[0]))

72

In [35]:
cas_inchi_df = list(dataset72_df1.index)
column_names = list(all_bonds_types_strs_short.keys())
target_df = pd.DataFrame(index = cas_inchi_df, columns = column_names)

In [36]:
#Iterate the rows
for ind, _ in enumerate(cas_inchi_df):
  try:
    target_df.iloc[ind, :] = find_cls_labels(dataset72_df1['peak_cls'].iloc[ind])
  except:
    None
target_df.dropna()

,C─H alkyl methyl 1260 strong/1380 weak/2870 medium to strong/2960 medium to strong,C─H alkyl methylene 1470 strong/2850 medium to strong/2925 medium to strong,C─H alkyl methine 2890 weak,C─H vinyl C═CH2 900 strong/2975 medium/3080 medium,C─H vinyl C═CH 3020 medium,C─H vinyl monosubstituted alkenes 900 strong/990 strong,C─H vinyl cis-disubstituted alkenes 670–700 strong,C─H vinyl trans-disubstituted alkenes 965 strong,C─H vinyl trisubstituted alkenes 800–840 strong to medium,C─H aromatic benzene/sub. benzene 3070 weak,...,C─X fluoroalkanes ordinary 1000–1100,"C─X fluoroalkanes trifluoromethyl 1100–1200 two strong, broad bands",C─X chloroalkanes any 540–760 weak to medium,C─X bromoalkanes any 500–600 medium to strong,C─X iodoalkanes any 500 medium to strong,N─O nitro compounds aliphatic 1540 stronger/aliphatic 1380 weaker,N─O nitro compounds aromatic 1520 lower/1350 lower if conjugated,P─C Organophosphorus compound aromatic 1440-1460 medium,P─O phosphorus oxide bonded 1195-1250 strong,P─O phosphorus oxide free 1250-1300 strong
7496-45-9,1,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,1,1,1,0
106-36-5,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,1,0,1,0
5347-82-0,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,1,1,1,0
112-17-4,1,1,0,0,0,0,1,1,1,0,...,1,0,0,1,0,0,1,1,1,0
4919-37-3,1,1,0,1,0,1,1,1,1,0,...,1,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527-69-5,1,1,0,1,1,0,0,1,1,0,...,0,0,1,0,0,0,1,1,0,0
92291-24-2,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5024-31-7,1,0,0,1,1,1,1,1,1,0,...,0,0,1,0,0,0,1,1,1,0
93861-86-0,0,1,1,0,0,0,0,0,1,1,...,0,0,0,0,0,1,1,0,1,0


# Оценка качества эмпирического алгоритма

In [37]:
classes = list(all_bonds_types_strs_short.keys())
y_true = []
y_pred = []
  
df = dataset72_df_labeled1.iloc[:,2:-6].dropna()
target_df1 = target_df.dropna()
for index, _ in target_df1.iterrows():
  try:
    y_true.append(torch.Tensor(df.loc[index]))
      
    y_pred.append(torch.Tensor(target_df.loc[index]))
  except:
      
      None
print(len(y_true), len(y_pred))
# print(y_true, y_pred)


9936 9936


In [38]:
from torch.nn.utils.rnn import pad_sequence
y_true1 = pad_sequence(y_true)
y_pred1 = pad_sequence(y_pred)
y_true1.T.shape

torch.Size([9936, 72])

In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_true1.T, y_pred1.T, target_names = classes))

                                                                                                                   precision    recall  f1-score   support

                               C─H alkyl methyl 1260 strong/1380 weak/2870 medium to strong/2960 medium to strong       0.62      0.90      0.73      6192
                                      C─H alkyl methylene 1470 strong/2850 medium to strong/2925 medium to strong       0.59      0.68      0.63      5877
                                                                                      C─H alkyl methine 2890 weak       0.25      0.22      0.24      2434
                                                               C─H vinyl C═CH2 900 strong/2975 medium/3080 medium       0.05      0.50      0.09       495
                                                                                       C─H vinyl C═CH 3020 medium       0.11      0.28      0.16      1130
                                                          C─H vinyl m

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Создание класса Dataset

In [40]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets

class IRSpectraDataset72(Dataset):
    def __init__(self, dataset_df_labeled):
        self.dataset_df = dataset_df_labeled
        self.spectra_labels = self.dataset_df.iloc[:,2:-6]
        self.spectra_x = self.dataset_df.iloc[:,-4]
        self.spectra_y = self.dataset_df.iloc[:,-3]
        self.spectra_y_peak = self.dataset_df.iloc[:,-2]
        
    def __len__(self):
        return len(self.dataset_df)

    def __getitem__(self, idx):
        
        spectra_x = torch.Tensor(self.spectra_x.iloc[idx]).reshape((1, 1800))
        spectra_y = torch.Tensor(self.spectra_y.iloc[idx]).reshape((1, 1800)) 
        spectra_y_peak = torch.Tensor(self.spectra_y_peak.iloc[idx]).reshape((1, 1800))     
        spectra = torch.cat((spectra_x, spectra_y, spectra_y_peak))
        
        label = torch.Tensor(self.spectra_labels.iloc[idx])
        #transform
        return spectra, label

In [41]:
dataset_ir_spectra = IRSpectraDataset72(dataset72_df_labeled1)

In [42]:
#Создание объекта класса и разбиение на трейн,тест и вал
from torch.utils.data import random_split
dataset_ir_spectra = IRSpectraDataset72(dataset72_df_labeled1.dropna())
train_size = int(0.7 * len(dataset_ir_spectra))
val_size = int(0.1 * len(dataset_ir_spectra))
test_size = len(dataset_ir_spectra) - train_size - val_size
train_dataset, test_dataset = random_split(dataset_ir_spectra, [train_size, int(test_size+val_size)])
test_dataset, val_dataset = random_split(test_dataset, [test_size, val_size])

In [43]:
#Создание загрузчиков
# Number of classes
classes = list(dataset_ir_spectra.spectra_labels.columns)
num_classes = len(classes)
num_classes

72

In [44]:
cls2inx = {}
inx2cls= {}
for i, classs in enumerate(classes):
  cls2inx[f'{classs}']= i
  inx2cls[i] = classs
#print(cls2inx)
#print(inx2cls)

In [45]:
# Расчёт весов
wts4cls = {}
for classs in classes:
  wts4cls[classs] = dataset_ir_spectra.spectra_labels[classs].mean()
#print(wts4cls)

In [47]:
#Веса классов
wts_list = []
for name, values in wts4cls.items():
  wts_list.append(values)
weight_for_every_class = torch.tensor(wts_list)
#print(weight_for_every_class)

In [48]:
# #Модифицирование весов для каждого класс:
# level = 0.3
# grain = 0.15
# lg = level/grain
# print(lg)
# #Классы меньше grain процентов - увеличиваются в level/20 раза 
# #Классы меньше 10 процентов - увеличиваются в level/10 раз 
# #Классы меньше 5 процентов - увеличиваются в level/5 раз
# #Классы меньше 1 процента - увеличиваются в level раз
# weight_for_every_class_modified = []
# for i in weight_for_every_class:
#   if i <= grain/8: weight_for_every_class_modified.append(i/(32/lg))
#   elif (i <= grain/4) and (i>grain/8): weight_for_every_class_modified.append(i/(16/lg))
#   elif (i <= grain/2) and (i>grain/4): weight_for_every_class_modified.append(i/(8/lg))
#   elif (i <= grain) and (i>grain/2): weight_for_every_class_modified.append(i/(4*lg))
#   else: weight_for_every_class_modified.append(i)
# weight_for_every_class_modified = torch.Tensor(weight_for_every_class_modified)
# print(weight_for_every_class)
# print(weight_for_every_class_modified)
# print(weight_for_every_class_modified.size())

In [49]:
#Расчёт веса образца на основе минимального веса любой функциональной группы 
labels1 = train_dataset.dataset.spectra_labels.to_numpy().copy()
labels1[labels1 == 0.0] = 100
#labels1

In [50]:
train_labels = torch.tensor(labels1)
weighted_labels  = train_labels*weight_for_every_class
weights_for_every_sample = []
for label_tensor in weighted_labels:
  weight = np.nanmin(label_tensor.numpy())
  weights_for_every_sample.append(weight)

weights_for_every_sample = torch.tensor(weights_for_every_sample)
print(weights_for_every_sample)

tensor([0.0540, 0.0540, 0.0540,  ..., 0.0519, 0.0540, 0.1094],
       dtype=torch.float64)


In [51]:
weights_train = weights_for_every_sample[:len(train_dataset)]

In [53]:
#Проверка на положительные вероятности
for i in range(len(weights_train)):
    if weights_train[i] < 0:
        weights_train[i] = abs(weights_train[i])

In [54]:
weights_train[weights_train<0]

tensor([], dtype=torch.float64)

In [55]:
#Сумма весов должна быть равна единице. Вычисление суммы:
sum_weights = 0
for i in range(len(weights_train)):
 
  #print(weights_train[i])
  if weights_train[i] == torch.tensor(np.nan, dtype=torch.float64) : weights_train[i] = 1
  #print(weights_train[i])
  #print(sum_weights)
  sum_weights +=weights_train[i].numpy()
print(sum_weights)

676.911948095711


In [56]:
#Сумма весов должна быть равна единице. Нормирование на сумму.
for i in range(len(weights_train)):
    weights_train[i] =  weights_train[i]/sum_weights

In [57]:
weights_train = weight_for_every_class

In [58]:
#Даталоадер

from torch.utils.data import DataLoader, WeightedRandomSampler

# Batch size
batch_size = 128

dataloader_ir_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
dataloader_ir_val = DataLoader(val_dataset, batch_size=val_size, shuffle=False)
weightedsampler = WeightedRandomSampler(weights_train, len(weights_train)) 

dataloader_ir_train = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    #sampler= weightedsampler
)

In [60]:
#Даталоадер для статистики

# Batch size
batch_size = 128
dataloader_ir_stat = DataLoader(dataset_ir_spectra, batch_size=batch_size, shuffle=False)

#Использование ML для классификации

In [61]:
dataset_df = dataset72_df_labeled1.dropna()
spectra_labels = dataset_df.iloc[:,2:-6]
spectra_y = dataset_df.iloc[:,-3]

In [62]:
spectra_labels1 = spectra_labels.to_numpy()
spectra_y1 = spectra_y.to_numpy()
spectra_labels1.shape

(11868, 72)

In [63]:
spectra_y3 = np.stack(spectra_y1)
spectra_y3.shape

(11868, 1800)

In [64]:
#Разбиение на трейн,тест и вал
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(spectra_y3, spectra_labels1, test_size=0.33, random_state=42, shuffle = True)

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
clf = RandomForestClassifier(n_jobs=-1, random_state=0, verbose=1, warm_start=True)
clf.fit(X_train, y_train)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.3min finished


RandomForestClassifier(n_jobs=-1, random_state=0, verbose=1, warm_start=True)

In [66]:
y_pred = clf.predict(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    1.6s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    4.0s finished


In [67]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names = classes))

                                                                                                                   precision    recall  f1-score   support

                               C─H alkyl methyl 1260 strong/1380 weak/2870 medium to strong/2960 medium to strong       0.82      0.90      0.86      2444
                                      C─H alkyl methylene 1470 strong/2850 medium to strong/2925 medium to strong       0.81      0.87      0.84      2319
                                                                                      C─H alkyl methine 2890 weak       0.73      0.31      0.43       962
                                                               C─H vinyl C═CH2 900 strong/2975 medium/3080 medium       0.96      0.32      0.48       201
                                                                                       C─H vinyl C═CH 3020 medium       0.93      0.25      0.40       443
                                                          C─H vinyl m

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Использование других методов ML


Затруднено использование Bagging в виду требования по размерности y(требуется 1d).

In [68]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split


def train_and_test_ML(model, data_x, data_y):
    X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.33, random_state=42, shuffle = True)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return classification_report(y_test, y_pred, target_names = classes)

In [69]:
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from tqdm import tqdm
models_zoo = {
    "DT": DecisionTreeClassifier(),
    "MLPClassifier": MLPClassifier()
}
result = {}
for name, model in tqdm(models_zoo.items()):
  result[name] = train_and_test_ML(model, spectra_y3, spectra_labels1)
  print(result[name])

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 50%|█████     | 1/2 [03:45<03:45, 225.38s/it]

                                                                                                                   precision    recall  f1-score   support

                               C─H alkyl methyl 1260 strong/1380 weak/2870 medium to strong/2960 medium to strong       0.78      0.80      0.79      2444
                                      C─H alkyl methylene 1470 strong/2850 medium to strong/2925 medium to strong       0.75      0.76      0.76      2319
                                                                                      C─H alkyl methine 2890 weak       0.42      0.43      0.42       962
                                                               C─H vinyl C═CH2 900 strong/2975 medium/3080 medium       0.42      0.47      0.44       201
                                                                                       C─H vinyl C═CH 3020 medium       0.36      0.40      0.38       443
                                                          C─H vinyl m

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and bei

                                                                                                                   precision    recall  f1-score   support

                               C─H alkyl methyl 1260 strong/1380 weak/2870 medium to strong/2960 medium to strong       0.84      0.87      0.85      2444
                                      C─H alkyl methylene 1470 strong/2850 medium to strong/2925 medium to strong       0.81      0.86      0.83      2319
                                                                                      C─H alkyl methine 2890 weak       0.64      0.42      0.50       962
                                                               C─H vinyl C═CH2 900 strong/2975 medium/3080 medium       0.70      0.42      0.52       201
                                                                                       C─H vinyl C═CH 3020 medium       0.63      0.39      0.48       443
                                                          C─H vinyl m

# Конец